This project explores different implementations of the negation operator in Champollion's semantics, and issues that come with each. 

In [1]:
import model_checker

from model_checker.theory_lib import exclusion
from model_checker.theory_lib.exclusion import examples



# Import operators
operators = exclusion.exclusion_operators
old_operators = exclusion.old_exclusion_operators

new_semantics = [exclusion.ExclusionSemantics, operators, exclusion.UnilateralProposition, exclusion.ExclusionStructure]
old_semantics = [exclusion.OldExclusionSemantics, old_operators, exclusion.OldUnilateralProposition, exclusion.OldExclusionStructure]

# Get default settings
default_settings = exclusion.ExclusionSemantics.DEFAULT_EXAMPLE_SETTINGS

# Define general settings for display
general_settings = {
    "print_constraints": False,
    "print_impossible": True,
    "print_z3": False,
    "save_output": False,
    "maximize": False,
}

# Update default settings with general settings
default_settings.update(general_settings)

In [2]:
def run_example(example, name, exclusion_version=new_semantics):
    """
    Run a specific example and display the results.
    
    Args:
        example: The example to run (list containing premises, conclusions, settings)
        name: The name of the example
    """
    premises, conclusions, settings = example
    semantics_class, operators, proposition_class, model_structure_class = exclusion_version
    
    # Create syntax object
    syntax = model_checker.syntactic.Syntax(premises, conclusions, operators)
    
    # Update default settings with example-specific settings and general settings
    example_settings = default_settings.copy()
    example_settings.update(settings)
    
    # Ensure print_impossible is set
    if 'print_impossible' not in example_settings:
        example_settings['print_impossible'] = True
    
    # Create semantics
    semantics = semantics_class(example_settings)
    
    # Create model constraints
    model_constraints = model_checker.model.ModelConstraints(example_settings, syntax, semantics, proposition_class)
    
    # Create model structure
    model_structure = model_structure_class(model_constraints, example_settings)
    
    # Interpret sentences before printing
    sentences = model_structure.premises + model_structure.conclusions
    model_structure.interpret(sentences)
    
    # Print results
    model_structure.print_all(example_settings, name, "Exclusion Semantics")

def run_example_range(example_range_dict, exclusion_version=new_semantics):
    for name, example in example_range_dict.items():
        run_example(example, name, exclusion_version)

As a baseline, we check what the old implementation of the semantics (conserved in old_semantics and old_operators) gives for the examples in examples.py:

In [4]:
# run all examples in the example range with the old semantics
run_example_range(examples.example_range, old_semantics)


EXAMPLE Only Frame Constraints: there is a countermodel.

Atomic States: 2

Semantic Theory: Exclusion Semantics


Z3 Run Time: 0.0028 seconds

State Space
  #b00 = □
  #b01 = a
  #b10 = b
  #b11 = a.b (world)

Coherence
  □ coheres with □
  □ coheres with a
  □ coheres with b
  □ coheres with a.b
  a coheres with □
  a coheres with a
  a coheres with b
  a coheres with a.b
  b coheres with □
  b coheres with a
  b coheres with b
  b coheres with a.b
  a.b coheres with □
  a.b coheres with a
  a.b coheres with b
  a.b coheres with a.b

Functions

The evaluation world is: a.b

Total Run Time: 0.1521 seconds


EXAMPLE No Gaps: there is a countermodel.

Atomic States: 3

Semantic Theory: Exclusion Semantics

Premise:

Conclusion:
1. (A \univee \exclude A)

Z3 Run Time: 0.0073 seconds

State Space
  #b000 = □
  #b001 = a
  #b010 = b (world)
  #b011 = a.b (impossible)
  #b100 = c
  #b101 = a.c (world)
  #b110 = b.c (impossible)
  #b111 = a.b.c (impossible)

Conflicts
  a conflicts with b
 

Now, see the new semantics below. This implementation quantifies over arrays. However, it is not perfect. Some of the models it finds are odd (there is not a single model it finds where an excluded sentence has a non-empty set as its proposition). In terms of classical entailments though, it is an improvement over the old semantics, for which countermodels to DeMorgan's entailments were found, which is not supposed to be the case for the exclusion semantics. 

In [ ]:
# run all examples in the example range with the old semantics
run_example_range(examples.example_range, new_semantics)

You can change the operator being used by going to `operators.py` in `exclusion` of `theory_lib` and changing the operator defined as `ExclusionOperator` at the bottom to whichever uses the `extended_verify` definition of your choosing. For a discussion of issues, see the taglines of the operators in `operators.py`, and comments in their `extended_verify` definitions. 

Many (most) of the classical theorems hold with the quantified arrays strategy. However, a notable example that does not is double negation elimination (though double negation introduction does hold):

In [5]:
run_example(examples.DN_ELIM_example, "Double Negation Elimination")


EXAMPLE Double Negation Elimination: there is a countermodel.

Atomic States: 3

Semantic Theory: Exclusion Semantics

Premise:
1. \exclude \exclude A

Conclusion:
2. A

Z3 Run Time: 0.1387 seconds

State Space
  #b000 = □
  #b001 = a
  #b010 = b
  #b011 = a.b (world)
  #b100 = c (world)
  #b101 = a.c (impossible)
  #b110 = b.c (impossible)
  #b111 = a.b.c (impossible)

Conflicts
  □ conflicts with a.c
  □ conflicts with a.b.c
  a conflicts with c
  a conflicts with a.c
  a conflicts with b.c
  a conflicts with a.b.c
  b conflicts with c
  b conflicts with a.c
  b conflicts with b.c
  b conflicts with a.b.c
  a.b conflicts with c
  a.b conflicts with a.c
  a.b conflicts with b.c
  a.b conflicts with a.b.c
  c conflicts with a
  c conflicts with b
  c conflicts with a.b
  c conflicts with a.c
  c conflicts with b.c
  c conflicts with a.b.c
  a.c conflicts with □
  a.c conflicts with a
  a.c conflicts with b
  a.c conflicts with a.b
  a.c conflicts with c
  a.c conflicts with a.c
  a.c 

This holds even if we set the example to be contingent, which in theory should require `\\exclude A` to have a nonempty proposition:

In [ ]:
# DOUBLE NEGATION ELIMINATION WITH CONTINGENCY
DN_ELIM_premises = ['\\exclude \\exclude A']
DN_ELIM_conclusions = ['A']
DN_ELIM_settings = {
    'N' : 3,
    'possible' : False,
    'contingent' : True,
    'non_empty' : False,
    'non_null' : False,
    'disjoint' : False,
    'fusion_closure' : False,
    'max_time' : 5,
    'iterate' : 1,
    'expectation' : True,
}
DN_ELIM_example = [
    DN_ELIM_premises,
    DN_ELIM_conclusions,
    DN_ELIM_settings
]

run_example(DN_ELIM_example, "Double Negation Elimination")

The biggest outstanding issue is making the printing line up with the satisfying of a model. For QA, Z3 finds models mostly when expected and doesn't when not. However these models remain odd, so the mismatch is likely in the printing. Threads to pursue include the function witnesses (credit to Ben—thank you for that idea!)